# Pre

In [1]:
# ! pip install pytorch_lightning;
# ! pip install wandb;

In [2]:
import wandb
import os
wandb.login()
wandb.__version__

wandb: Currently logged in as: gsaltintas (use `wandb login --relogin` to force relogin)


'0.12.11'

In [3]:
# Rodrigo Caye Daudt
# rodrigo.cayedaudt@geod.baug.ethz.ch
# 04/2021

%matplotlib inline
import numpy as np
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import torch
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


from EuroSAT_dataset import EuroSAT
from network import Net


%load_ext autoreload
%autoreload 2


if not os.path.exists('./outputs'):
    os.mkdir('./outputs')

print('Imports OK')

ModuleNotFoundError: No module named 'skimage'

# Global Parameters

In [5]:
# Global parameters

# If USE_CUDA is True, computations will be done using the GPU (may not work in all systems)
# This will make the calculations happen faster
USE_CUDA = torch.cuda.is_available() 

DATASET_PATH = '../EuroSAT_data'

BATCH_SIZE = 32 # Number of images that are used for calculating gradients at each step

NUM_EPOCHS = 20 # Number of times we will go through all the training images. Do not go over 25

LEARNING_RATE = 1e-3 # Controls the step size
MOMENTUM = 0.9 # 0.0 # Momentum for the gradient descent
WEIGHT_DECAY = 1e-6 # Regularization factor to reduce overfitting


grading = False
print('Parameters OK')

NameError: name 'torch' is not defined

In [4]:
from torch._C import device
# Create datasets and data loaders
train_dataset = EuroSAT(DATASET_PATH, True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

test_dataset = EuroSAT(DATASET_PATH, False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


print('Dataloaders OK')

/home/gsa/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'EuroSAT' is not defined

# Training

In [8]:
# Criterion, optimizer, and scheduler

criterion = torch.nn.CrossEntropyLoss() # Do not change this
# optimizer = torch.optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using', device)

# Create network
net = Net(loss_func=criterion, lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
if USE_CUDA:
    net = net.cuda()

print('Network OK')

Using cpu
Network OK


In [13]:
net.device, device

(device(type='cpu'), device(type='cpu'))

In [14]:
# Define pytorchlightning parameters
grading = True
if not grading: 
    wandb_logger = WandbLogger(entity=os.environ['WANDB_ENTITY'],
                           project='remote_sensing',
                           log_model=True
                           )
    wandb_logger.watch(net)
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
    )
    lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
    trainer_params = {'callbacks': [checkpoint_callback, lr_monitor], 
                    'max_epochs': NUM_EPOCHS,
                      'auto_lr_find': True,
                      'logger': wandb_logger,
                      'log_every_n_steps': 5
                     }
else:
    trainer_params = {'max_epochs': NUM_EPOCHS,
                      'auto_lr_find': True,
                      'logger': False,
                     }
if USE_CUDA:
    trainer_params['gpus'] = [0]
    
trainer = pl.Trainer(**trainer_params)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(net, train_loader, test_loader,)

# Testing

In [15]:
# Helper function to organize main loop
# This function is called for training and for testing at each epoch

def run_epoch(net, optimizer, dataloader, criterion, train=True, cuda=USE_CUDA):
    epoch_total_loss = 0
    epoch_total_samples = 0
    epoch_total_correct = 0

    for sample in tqdm(dataloader):
        img = sample['image']
        label = sample['label']

        if cuda:
            img, label = img.cuda(), label.cuda()

        if train:
            optimizer.zero_grad()
        out = net(img)
        loss = criterion(out, label)
        if train:
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            epoch_total_samples += img.size(0)
            epoch_total_loss += img.size(0) * loss
            epoch_total_correct += torch.sum(torch.argmax(out, dim=1) == label)

    return epoch_total_loss / epoch_total_samples, epoch_total_correct / epoch_total_samples


In [17]:
net.eval()
loss, accuracy = run_epoch(net, None, test_loader, criterion, False)
print(f'Final test loss: {loss}')
print(f'Final test accuracy: {accuracy}')

100%|█████████████████████████████████████████| 169/169 [00:25<00:00,  6.68it/s]


(tensor(2.3687), tensor(0.0919))